In [1]:
import pandas as pd
import os
import json
import numpy as np

In [2]:
ASSETS_DIR = "/media/hdd/usr/edo/egoProcel_mistakes/assets"
DATAFOLDER = "/home/aleflabo/ego_procedural/OadTR/data/EpicTents/adapted_annotations/mistake_annotations"
ALL_CSV_NAMES = os.listdir(DATAFOLDER)
TRAIN_SPLIT = (
    "/home/aleflabo/ego_procedural/OadTR/data/EpicTents/OadTR/data_info_new.json"
)

In [6]:
j = json.load(open(TRAIN_SPLIT))
CORRECT_PROC = j["epic_tents"]["train_session_set"]

In [7]:
len(CORRECT_PROC)

14

In [8]:
IDXS_ACTIONS = "/home/aleflabo/ego_procedural/OadTR/data/EpicTents/OadTR/mapping.txt"
idxs_actions = np.loadtxt(IDXS_ACTIONS, dtype=str)
idx2action = {int(idx): action for idx, action in idxs_actions}
action2idx = {action: int(idx) for idx, action in idxs_actions}

In [32]:
out = {}
for csv_name in CORRECT_PROC:
    csv_path = os.path.join(DATAFOLDER, f"{csv_name}.csv")
    df = pd.read_csv(csv_path)

    s_num = df["action_label"].values.tolist()
    s_alpha = list(map(lambda x: idx2action[x], s_num))

    out[csv_name] = {"alpha": s_alpha, "num": s_num}

In [33]:
json.dump(out, open(os.path.join(ASSETS_DIR, "epictents_context_seqs_train.json"), "w"))

In [34]:
def get_prompt(seq):
    hist = [-1] + seq[:-1]
    next_sym = seq[-1]
    prompt = f"Input Sequence:\n {', '.join(map(str,hist))}\n"
    prompt += f"Next Symbol:\n {next_sym}\n---\n"
    return prompt

In [36]:
prompt_num, prompt_alpha = "", ""
for k, v in out.items():
    prompt_num += get_prompt(v["num"])
    prompt_alpha += get_prompt(v["alpha"])
out_context = {"num": prompt_num, "alpha": prompt_alpha}

In [39]:
json.dump(
    out_context,
    open(os.path.join(ASSETS_DIR, "epictents_context_prompt_train.json"), "w"),
)